In [1]:
import pandas as pd
import yfinance as yf
from ta.volatility import average_true_range

In [2]:
# Load and preprocess data
def preprocess_data():
    data = yf.download("EURUSD=x", period="5d", interval="1h")
    data.index = pd.to_datetime(data.index)
    data = data.drop(columns=['Adj Close', 'Volume'])
    data

In [3]:
# Calculate SuperTrend
def supertrend(data):  
    # Calculate ATR
    data['atr'] = average_true_range(data['High'], data['Low'], data['Close'], window=10)
    
    # Calculate upper and lower bands
    data['upperband'] = ((data['High'] + data['Low']) / 2) + (3 * data['atr'])
    data['lowerband'] = ((data['High'] + data['Low']) / 2) - (3 * data['atr'])
    
    # Initialize SuperTrend
    data['supertrend'] = 0.0
    data['supertrend'][0] = data['upperband'][0]
    
    # Calculate SuperTrend
    for i in range(1, len(data)):
        if data['Close'][i] > data['upperband'][i-1]:
            data['supertrend'][i] = data['upperband'][i]
        elif data['Close'][i] < data['lowerband'][i-1]:
            data['supertrend'][i] = data['lowerband'][i]
        else:
            data['supertrend'][i] = data['supertrend'][i-1]
    
    return data

In [4]:
# Define the trading strategy
def trading_strategy(data):
    position = 0  # 0 means no position, 1 means long, -1 means short
    entry_price = 0

    for i in range(1, len(data)):
        if position == 0:
            # Check for long entry
            if data['supertrend'][i] > data['Close'][i]:
                position = 1
                entry_price = data['Close'][i]
                print(f"Entering long at {entry_price} on {data.index[i]}")
            # Check for short entry
            elif data['supertrend'][i] < data['Close'][i]:
                position = -1
                entry_price = data['Close'][i]
                print(f"Entering short at {entry_price} on {data.index[i]}")
        elif position == 1:
            # Check for long exit
            if data['supertrend'][i] < data['Close'][i]:
                position = 0
                cash += (data['Close'][i] - entry_price) * 10  # Assuming we buy/sell 10 units
                highest_value = max(highest_value, cash)
                lowest_value = min(lowest_value, cash)
                print(f"Exiting long at {data['Close'][i]} on {data.index[i]}")
        elif position == -1:
            # Check for short exit
            if data['supertrend'][i] > data['Close'][i]:
                position = 0
                cash += (entry_price - data['Close'][i]) * 10  # Assuming we buy/sell 10 units
                highest_value = max(highest_value, cash)
                lowest_value = min(lowest_value, cash)
                print(f"Exiting short at {data['Close'][i]} on {data.index[i]}")
        
        data.at[data.index[i], 'position'] = position
    
    return data

In [5]:
# Run the backtest
if __name__ == "__main__":
    data = preprocess_data()
    data = supertrend(data)
    results = trading_strategy(data)
    


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['EURUSD=X']: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='fc.yahoo.com', port=443): Read timed out. (read timeout=30)"))


TypeError: 'NoneType' object is not subscriptable